# Random Projection on PYNQ-Z1
### n_features = 128, n_components=32

In [1]:
import numpy as np
from sklearn import datasets
from pynq_sklearn.random_projection import PynqBinaryRandomProjection
import time


N = 10
n_features = 128
n_components = 32

np.random.seed(23)
x = np.random.rand(N*n_features).astype(np.float32)*2 - 1
X = x.reshape(-1, n_features)


# Generate dataset of Ints
X, y = datasets.make_regression(n_samples=5000, n_features=128, n_targets=32, random_state=43, noise=4.0,
                       bias=26.0)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1000, random_state=42)

#### 1. Fit a PYNQ Linear Regression Model

In [2]:
model = PynqBinaryRandomProjection()
model.fit(X_train, y_train)

PynqBinaryRandomProjection()

In [3]:
print(model.components_)

[[ 0.1767767 -0.1767767 -0.1767767 ...  0.1767767  0.1767767 -0.1767767]
 [ 0.1767767 -0.1767767 -0.1767767 ... -0.1767767  0.1767767 -0.1767767]
 [ 0.1767767 -0.1767767  0.1767767 ... -0.1767767  0.1767767 -0.1767767]
 ...
 [ 0.1767767 -0.1767767  0.1767767 ... -0.1767767 -0.1767767  0.1767767]
 [ 0.1767767  0.1767767  0.1767767 ...  0.1767767 -0.1767767  0.1767767]
 [ 0.1767767 -0.1767767 -0.1767767 ... -0.1767767 -0.1767767 -0.1767767]]


#### 2. With hw_accel=True, deploy prediction on the FPGA 

In [5]:
# prepare X_test and y_test for HW
FRAC_WIDTH = 20
X_test_hw = (X_test*(1<<FRAC_WIDTH)).astype(np.int32)
y_test_hw = (y_test*(1<<FRAC_WIDTH)).astype(np.int32)

X_test_hw = model.copy_array(X_test_hw, dtype=np.int32) # allocates X_test_hw to contiguous memory
print(X_test_hw)

[-1822086 -1210398  -715734 ...  -234639   727398  -678209]


In [6]:
y_pred = model.transform(X_test_hw)

#### 3. Alternatively, deploy in SW

In [7]:
model.hw_accel=False
y_pred_sw = model.transform(X_test)

#### 4. Verify equivalence

In [9]:
ypred = y_pred*(1.0/(1<<FRAC_WIDTH))
print(ypred-y_pred_sw)

[[-6.27675036e-06  2.34110358e-05 -6.39649152e-06 ... -2.52877929e-06
   1.55040112e-05  1.64285986e-05]
 [ 5.08022788e-06  1.34924156e-05  7.19602352e-06 ... -1.55459307e-05
  -1.16528528e-06 -5.86435664e-06]
 [-1.30007541e-05  2.31459193e-05 -8.72852779e-06 ... -2.31959959e-06
   1.55894037e-05  1.79093341e-06]
 ...
 [ 7.31469715e-06 -8.06458421e-06 -2.54194398e-05 ... -3.99087231e-07
  -9.21467164e-08  1.80637067e-06]
 [ 7.59719099e-06 -4.95562029e-06  5.87763739e-06 ...  1.15208674e-06
  -8.24086042e-06  2.24593795e-06]
 [ 4.44624937e-06  2.64261518e-07  1.65942603e-05 ...  1.41842695e-07
  -2.03101093e-05  6.64835247e-06]]


#### 5. Performance
Bad performance observed because of sw-to-hw processing in lin.predict()

In [11]:
import timeit
number=200

model.hw_accel=True
def hwresp():
    model.transform(X_test_hw) #, out=outBuffer)

hw_time = timeit.timeit(hwresp,number=number)

model.hw_accel=False
def swresp():
    model.transform(X_test)

sw_time = timeit.timeit(swresp,number=number)

print("Time taken by sklearn", number,"times",sw_time)
print("Time taken by sklearn+fpga", number,"times",hw_time)
print("HW Speedup = %dx"%(sw_time/hw_time))

Time taken by sklearn 200 times 6.819886854000288
Time taken by sklearn+fpga 200 times 0.38277368299986847
HW Speedup = 17x


### Verify Equivalence Again

In [12]:
N = 10
n_features = 128
n_outputs = 32

np.random.seed(23)

x = np.random.rand(N*n_features).astype(np.float32)*2 - 1
x_hw = (x*(1<<FRAC_WIDTH)).astype(np.int32)
x_sw = x.reshape(-1, n_features).astype(np.float32)

#hw accelerator
model.hw_accel = True
xtest = model.copy_array(x_hw, dtype=np.int32)
ypred = model.transform(xtest)
ypred = ypred*(1.0/(1<<FRAC_WIDTH))

#sw
model.hw_accel = False
ypred_sw = model.transform(x_sw)

In [13]:
print(ypred - ypred_sw)

[[ 1.83527915e-06 -2.13944442e-06 -5.84024172e-06 -6.02874820e-06
   4.62859838e-06  7.20564891e-06  1.60893303e-05  4.11545073e-06
  -2.52978283e-06  5.59115614e-06 -1.06071358e-05  7.97862605e-06
   1.85282097e-06  3.23806411e-06 -2.53164524e-06  5.94860143e-06
  -3.64340795e-08  1.06124746e-06  4.60918666e-06 -6.94521485e-06
   4.44943116e-06 -3.12234613e-06 -1.61616836e-06 -1.07983270e-05
   1.63545324e-06 -4.28921112e-06 -1.29988493e-07  2.57533403e-06
   1.04310961e-06  2.36485672e-06 -4.60611102e-06 -9.78871793e-06]
 [ 2.36327006e-06 -3.72025301e-06  1.52394757e-06  3.04487167e-06
   1.91724378e-06 -9.45277674e-07  3.82111312e-07 -5.12143936e-06
  -8.65333176e-06 -9.83656179e-06  5.31111240e-06  2.72797670e-06
  -7.10063905e-06  9.61562698e-06  1.45431030e-06  8.50742076e-06
   8.79894623e-06  2.17341779e-06  6.40424839e-06 -2.38438427e-07
  -4.54262165e-06  3.52338778e-06 -4.10645998e-06 -2.24059376e-06
   6.20659869e-06 -1.77645678e-07 -3.43337176e-07 -8.90351738e-06
  -7.6394

In [14]:
print(ypred)

[[-0.27790737  0.52748585  0.98072529  1.02838039 -1.10417557 -1.71545506
  -3.18398285 -1.04175472  0.44072533 -1.01846981  2.11191654 -1.62595081
  -0.5644474  -0.66141605  0.23148727 -1.52308273  0.10831356 -0.63661671
  -0.7220192   1.08036709 -0.65278816  0.67103195  0.24839306  2.17098904
  -0.54709911  0.78272057 -0.08471775 -0.34614468 -0.1537838  -0.842659
   0.8308506   1.8434782 ]
 [-0.766078    0.4579401  -0.30020523 -0.61380482 -0.39260101 -0.06364536
  -0.32569122  0.65054131  1.50471592  1.7324028  -1.32352924 -0.49001598
   1.32403851 -1.66122913 -0.48660088 -1.75505543 -1.83498859 -0.54886913
  -1.16464138  0.13675499  0.66380215 -0.92720032  0.90714264  0.38257027
  -1.35356712  0.01125908  0.11333179  1.52713299  1.50508308 -0.61142826
   1.50432014  1.40013695]
 [-0.70541096 -0.38744068  0.17312145 -0.07317924 -0.79022503  1.9304781
   2.28448486  2.2528429   0.16735077 -0.58813     0.76954746 -0.21823215
   1.01915073  0.23613071 -2.14978409 -0.79039288 -0.30968285

In [15]:
print(ypred_sw)

[[-0.27790921  0.52748799  0.98073113  1.02838642 -1.1041802  -1.71546226
  -3.18399894 -1.04175884  0.44072786 -1.0184754   2.11192715 -1.62595879
  -0.56444926 -0.66141929  0.23148981 -1.52308868  0.1083136  -0.63661777
  -0.7220238   1.08037403 -0.65279261  0.67103507  0.24839467  2.17099983
  -0.54710075  0.78272486 -0.08471762 -0.34614725 -0.15378484 -0.84266136
   0.83085521  1.84348799]
 [-0.76608036  0.45794382 -0.30020675 -0.61380786 -0.39260293 -0.06364442
  -0.32569161  0.65054643  1.50472457  1.73241264 -1.32353455 -0.49001871
   1.32404561 -1.66123875 -0.48660233 -1.75506393 -1.83499739 -0.54887131
  -1.16464778  0.13675523  0.66380669 -0.92720384  0.90714675  0.38257251
  -1.35357333  0.01125926  0.11333214  1.52714189  1.50509072 -0.61143032
   1.50432765  1.40014528]
 [-0.70541427 -0.38744241  0.17312174 -0.07318039 -0.79022734  1.93048822
   2.28449663  2.25285377  0.16735131 -0.58813169  0.76955107 -0.21823202
   1.01915585  0.23613187 -2.14979429 -0.79039711 -0.30968